In [1]:
## remember to run conn_cocolab from the terminal before running cells in this notebook!
import os
import urllib, cStringIO

import pymongo as pm
import cv2
import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')

import numpy as np
from __future__ import division
import scipy.stats as stats
import pandas as pd
import json
import re

from PIL import Image
import base64
import datetime

### File hierarchy and database connection vars 

In [10]:
# directory & file hierarchy
iterationName = 'cdm_run_v3'
exp_path = 'museumstation'
analysis_dir = os.getcwd()
exp_dir = os.path.abspath(os.path.join(os.getcwd(),'../..','experiments'))
sketch_dir = os.path.join(analysis_dir,'sketches')
data_dir = os.path.join(analysis_dir,'usage_data')
scribble_dir = os.path.join(analysis_dir,'scribbles')

# load data
scribble_file = os.path.join(data_dir, "scribble_annotate.csv")
raw_data = pd.read_csv(scribble_file)


In [11]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['kiddraw']
coll = db['cdm_run_v3']


In [ ]:
# A list to store scribbles eliminated from hard standards
# 1. num_strokes> 100
# 2. prop_black > 0.9
scribble_list = []
scribble_list = raw_data.loc[raw_data[num_strokes>100]

### Get features and preprocessing

#### Proportion of black pixels on the png image

In [12]:
# Get pixel proportions on each sketch
raw_data['num_black_pixels'] = 0
raw_data['prop_black'] = 0.0

for index, row in raw_data.iterrows():
    fname = row.filename
    img_name = fname.split('/')[-1]
    cat_name = img_name.replace('.png','').split('_')[0]
    fpath = os.path.join(scribble_dir, cat_name, img_name)
    
    sketch = cv2.imread(fpath, cv2.IMREAD_UNCHANGED)
    
    # get the number of all black pixels
    pixels_black = np.argwhere(sketch.sum(axis=2) > 0)
    num_black_pixels = len(pixels_black)
    
    # the number of all pixels
    num_all_pixels = len(sketch) * len(sketch[0])  # row * col
    
    raw_data.at[index, 'num_black_pixels'] = num_black_pixels
    raw_data.at[index, 'prop_black'] = float(num_black_pixels)/float(num_all_pixels)


#### Stroke length

In [43]:
import svg_render_helpers as rdh
from scipy.spatial import distance
reload(rdh)

# Get svg data 
raw_data['max_stroke_length'] = 0
raw_data['min_stroke_length'] = 0
raw_data['avg_stroke_length'] = 0
stroke_data = []


for index, row in raw_data.iterrows():
    
    # get stroke data for the current drawing
    stroke_recs = coll.find({'$and': [{'sessionId':row.sessionId}, 
                                      {'dataType':'stroke'},
                                      {'category': row.category}]}).sort('time')  
    try:
    ## check to make sure that there is at least one stroke! 
        if stroke_recs.count()>0:
            
            # if there are more than max_num of strokes, only include the first max_num of strokes
            max_num = 50
            if stroke_recs.count()> max_num:
                stroke_recs = stroke_recs[:max_num]

            ## make an svg list of all strokes
            svg_list = rdh.make_svg_list(stroke_recs)
            
            ## convert all strokes into two lists
            ## Verts includes vertice infor of each stroke
            ## Codes includes coordinates infor of each stroke
            Verts, Codes = rdh.get_verts_and_codes(svg_list)
            
            stroke_length_list = []
            
            # iterate each strokeo
            for verts in Verts:
                stroke_length = 0
                
                # sum the distance between each two continuous vertices
                for v_index, vertex in enumerate(verts[:-1]):
                    stroke_length += distance.euclidean(np.array(vertex),np.array(verts[v_index+1]))
                
                stroke_length_list.append(stroke_length)
                print index, stroke_length
            
            raw_data.at[index, 'max_stroke_length'] = np.max(stroke_length_list)
            raw_data.at[index, 'min_stroke_length'] = np.min(stroke_length_list)
            raw_data.at[index, 'avg_stroke_length'] = np.average(stroke_length_list)
            
    except Exception as e:
            print 'Oops, something went wrong! Here is the error:'  
            print e
            pass

0 1169.13824021
0 1698.5261658
0 2103.38667098
0 2742.15005455
0 3215.32352887
0 3575.2483102
0 4544.818574
0 5462.11005449
1 1289.8809472
1 2029.01138772
1 2384.62704721
1 2718.93758713
2 1495.30154016
2 2025.86908339
2 2734.83131968
2 5186.67611615
3 1638.23937806
3 2194.27509112
3 2660.34025108
3 3490.06146027
4 1210.65073472
4 1711.67130371
4 1711.67130371
4 3797.06795086
4 4049.35103184
4 4049.35103184
4 4049.35103184
4 4862.37067452
4 5085.91874608
4 5204.88512789
4 6126.21186064
4 6349.7599322
4 6468.72631401
4 7390.05304675
4 7613.60111832
4 7732.56750012
4 9829.49475727
4 11533.3045212
4 12087.4046781
4 13956.5497058
4 14656.4051441
4 14656.4051441
4 16656.6623933
4 17452.3996854
4 17452.3996854
4 17452.3996854
4 19592.0963779
4 20509.0774912
4 20509.0774912
4 20509.0774912
4 20509.0774912
4 22641.32226
4 23558.3033733
4 23558.3033733
4 23558.3033733
4 23558.3033733
4 24364.8663206
4 25648.469429
4 26487.5401873
4 27763.9638352
4 28603.0345935
5 245.219747377
5 860.37747452
5 

49 1538.33679804
49 2082.07765678
49 2822.63605477
49 3362.66700396
49 3557.34575315
50 1458.54901182
50 5575.76207698
51 1399.00553116
51 2676.38564167
51 2893.45251955
51 3160.96546633
51 3160.96546633
51 3415.29250763
51 3646.57364562
51 3730.45395137
51 4276.02363599
51 4594.10881214
51 4594.10881214
51 5176.46515535
51 5346.91456134
51 5960.82808482
51 6131.27749081
52 596.107685162
52 2162.95686488
52 3340.06138047
52 4127.5671127
52 4746.83789599
52 4927.32612301
52 5039.47541562
52 5137.75677404
52 5236.65928022
52 5332.82179343
52 5525.71501847
52 5570.32056975
52 5677.18230639
53 1754.10014945
53 1955.51789875
53 2562.68721556
53 3428.95128658
53 4526.11360878
53 4526.11360878
53 4950.39872886
53 5049.7893435
53 5155.59763568
53 5591.80457177
53 5591.80457177
53 5677.00576224
53 5718.40631169
53 5808.90405657
54 617.011539055
55 143.544418213
55 143.544418213
55 759.102376927
55 1087.55255838
55 1337.55435868
55 1337.55435868
55 2099.60582774
55 2428.05600919
55 2678.05780949

99 467.582781518
99 1021.95806335
99 1518.71670998
99 1921.02206102
99 2545.36747995
100 936.505897014
100 1348.9517562
100 1922.18968828
100 2304.94740489
101 1663.62558863
101 1701.85281602
101 1701.85281602
101 1756.96182725
101 1924.84186206
101 2045.55685693
101 2257.28336852
101 2257.28336852
101 2786.07506911
101 3311.78061012
101 3813.11319899
101 4583.35399739
101 5309.14566335
101 5897.37707685
102 1909.55750182
102 2367.56293731
102 2367.56293731
103 305.559651495
103 1015.17351855
103 1920.61509521
103 2108.40274517
103 2921.4955024
103 3863.68343594
104 1541.50710878
104 2135.6519145
104 4099.17663927
104 5135.35913295
104 5135.35913295
104 7189.86308154
104 8226.04557522
104 8226.04557522
105 1371.33505742
105 1468.23915228
105 2149.317256
105 2594.47348794
105 3369.11637869
105 4043.61537958
106 6720.44907074
106 12683.1505535
107 846.982939589
107 2027.88768257
107 2893.0063791
107 3867.01408925
107 4776.7034772
107 5359.60071109
107 5594.78751648
107 5594.78751648
107 

150 5384.79304118
150 5595.90175781
150 5595.90175781
150 5595.90175781
150 5749.32670964
151 712.53349545
151 1360.26359361
151 2714.02430526
151 3948.42417376
151 5093.23148208
152 1403.61857844
152 2397.70919146
152 3497.04811556
152 4063.54631907
152 4814.76437609
152 5554.62005152
152 6021.93144341
152 6413.98031687
152 6785.0977065
153 0
153 1572.78596733
153 3231.74793802
153 4024.3370098
154 983.522809379
154 2385.86164247
154 2991.38030239
154 3707.16884021
154 4319.84262038
154 5156.80409142
154 5376.65415068
154 5690.22060576
154 6091.95346177
154 6590.04867685
154 6943.16358544
154 8300.84065604
154 8951.70211809
155 2297.5624742
155 3907.71668361
156 820.639427807
156 2049.78933047
156 2659.76265902
156 3866.80894697
157 563.203042346
157 1698.11427265
157 3818.5715022
158 2097.74971251
158 2552.38746321
158 3116.20537725
158 3116.20537725
158 3116.20537725
158 3116.20537725
158 3704.86856944
159 371.459192032
159 1031.01632816
159 1590.40276943
159 2412.73370394
159 3718.

194 1373.54160349
194 2811.90216535
194 3879.38176392
194 5179.72890677
194 5472.24525852
194 5849.64203499
195 176.534733105
195 790.013940632
195 1367.46960862
195 2170.48683282
195 2749.58254425
195 3821.63748895
195 4084.11509197
195 4084.11509197
195 4249.38108747
195 4646.21352662
195 5933.98330989
195 6249.20505182
196 0.0
196 0.0
196 0.0
196 0.0
196 23.8309518948
196 420.277965905
196 1554.85847056
196 2897.17315243
197 41.5110970282
197 270.577771527
197 486.241100812
197 715.30777531
198 3392.71631991
198 6173.92023246
198 9854.15465382
198 12705.2369606
198 15537.5070826
198 19134.6475799
198 22042.4226317
198 25574.3174943
198 30077.290753
198 32947.3921382
198 35314.5721864
198 37815.8773023
198 39084.7433012
198 39084.7433012
198 39084.7433012
198 39084.7433012
198 39084.7433012
198 39084.7433012
198 39084.7433012
198 39084.7433012
198 39084.7433012
198 39084.7433012
198 39084.7433012
198 39084.7433012
198 39084.7433012
198 39084.7433012
198 43338.9060791
198 46120.109991

...

613 4962.47095513
613 4962.47095513
613 5719.26099394
613 5936.21759339
613 6258.22380451
613 6637.12528237
613 6959.13149349
613 7731.88744911
614 673.973460461
614 991.071184853
614 1177.40637692
614 1505.11176726
614 1743.61403931
614 1910.24970005
614 2065.94836365
614 2065.94836365
614 2235.30188389
614 2533.98297373
614 2533.98297373
614 2918.49770333
614 3116.13010632
614 4876.45803194
614 6906.64243933
614 7283.4280344
615 239.958128759
615 685.876983746
615 1639.74411466
615 2098.73272671
615 2395.02637138
615 2792.54576338
615 2792.54576338
615 2792.54576338
615 3328.92637623
615 3328.92637623
615 3328.92637623
615 3457.70545154
615 3685.40905194
616 802.60758006
616 1094.30196757
616 1094.30196757
616 1094.30196757
616 1272.97984162
616 1419.4782207
616 1809.24073196
616 2460.78703796
616 3198.27236555
616 3404.84347375
616 3808.53434565
616 4288.56509724
616 4701.67187356
617 647.826015099
617 1574.40430453
617 2362.31449437
617 3425.66413344
617 4536.77406084
617 4945.5432

643 14465.6832937
643 16550.7526004
643 17167.7777931
643 17431.7981394
643 17496.9901635
643 17846.7747501
643 18110.7950964
643 18175.9871205
643 20253.9393803
643 20638.7522092
643 22734.3389697
643 23119.1517985
644 1282.36791642
644 1598.08068759
644 1821.9768151
644 2075.97046612
644 2628.02297996
644 2628.02297996
644 2737.92210804
644 3464.34320588
644 3742.80762964
644 4099.62757035
644 4371.09884887
644 4645.54077963
644 4800.99585916
644 5045.53517877
644 5214.69930585
644 5671.44545889
644 5786.96835472
644 5972.02610017
644 6206.6576149
644 6584.02396709
644 6762.63611633
644 6839.73471192
644 6991.79602471
644 7162.10167932
644 7558.80339199
644 7656.85641435
644 7843.02977446
644 8005.97135856
645 2236.06969181
645 3640.14537641
645 5660.32846581
645 7560.06363364
645 9333.33317371
645 10157.2037719
645 10157.2037719
645 10157.2037719
645 10157.2037719
645 11707.303087
645 12664.0238808
645 13358.6148197
645 13862.8675988
645 14012.4503521
645 14778.3477697
645 14988.200

673 779.444653513
673 1138.19214683
673 2070.98400623
673 2446.10911373
673 3613.8637049
673 3933.00971027
673 4788.77147516
673 4969.6255309
673 6780.96518744
674 2692.53659506
674 4224.55943424
674 4385.34928622
674 4793.55552149
674 5388.11770162
675 709.251513849
675 1180.50722996
675 1384.57902309
675 1697.75915225
675 1697.75915225
675 2216.79360829
675 2675.67182647
675 3466.35897957
675 3925.23719775
675 5277.98157011
675 5642.42477712
675 6113.28813977
675 6771.26063782
675 7343.94434496
675 7654.68537792
675 8398.09043807
675 9048.51391813
675 9615.87295138
675 9823.8040404
675 9932.16722685
676 561.612699877
676 1004.55141933
676 2389.68912155
676 2890.61348269
676 2890.61348269
676 4042.1548621
676 4513.19729648
676 4513.19729648
676 4513.19729648
676 5656.70731855
676 6127.74975293
676 6127.74975293
676 6127.74975293
676 6504.0462659
676 6699.39293215
676 7059.43317755
676 7327.42089267
676 7739.13831645
676 8007.12603157
676 8751.90515064
677 478.705527387
677 535.3911137

705 299.920371646
705 1071.07307105
705 3494.85433044
705 4062.26205297
705 4506.32124932
705 4883.8524164
705 5390.7066263
705 5664.81632471
706 480.37856023
706 1691.41272812
706 2092.97065473
706 2667.56445632
706 4189.70965014
706 4549.49251413
706 4739.53822215
706 4988.58071672
706 5263.2119118
706 5379.24110932
706 5548.48650636
706 6249.06554951
707 1726.23347731
707 2172.90883968
707 4113.62075413
707 4560.2961165
707 5263.84834624
707 5695.46391593
707 6292.53317391
707 6724.1487436
707 8088.94895844
707 9005.02751268
707 9093.06319896
707 9326.23332209
708 543.388296603
708 1801.63765054
708 2051.00905999
708 2051.00905999
708 2323.64496442
708 2584.55491057
708 2753.98041042
708 2828.31075415
708 3076.83765972
708 3337.74760587
708 3507.17310572
708 3581.50344945
708 3843.18330145
708 4125.98712059
708 4377.68713478
708 4377.68713478
708 4377.68713478
708 4639.36698678
708 4922.17080592
708 5173.87082012
708 5173.87082012
708 5173.87082012
709 476.822327911
709 931.14425432

755 4197.88730514
755 5146.61507188
756 16.7213493517
756 84.9458160171
756 175.545443638
756 243.769910303
756 2300.98810812
757 1590.57380807
757 2036.64855658
757 2607.79460829
758 3208.90211924
758 5551.98246352
758 6149.06246993
758 6734.30222127
759 2503.9539877
759 2720.34483778
759 6003.17069856
759 6620.1572488
759 6620.1572488
759 6620.1572488
759 10214.5807696
759 11462.9227933
759 12288.5775201
759 12288.5775201
759 12288.5775201
759 15952.0928768
759 17336.063107
759 18331.8282672
759 18575.7495332
759 18575.7495332
759 18575.7495332
759 22180.0271651
759 23764.6035918
759 25102.4765173
759 25606.3358798
759 25606.3358798
759 25606.3358798
759 25606.3358798
759 25606.3358798
759 29380.7019834
759 30977.5560895
759 32359.9157264
759 33011.5061008
759 33083.22949
759 33083.22949
759 33083.22949
759 33083.22949
759 33083.22949
759 37014.7945227
759 38611.6486289
759 39994.0082658
759 40645.5986402
759 40717.3220294
759 40717.3220294
759 40717.3220294
759 40717.3220294
759 407

In [44]:
print raw_data['max_stroke_length']

0       5462
1       2718
2       5186
3       3490
4      28603
5       1985
6       6050
7        641
8       2362
9       3841
10      3255
11      4994
12      9135
13      1508
14      5699
15      1716
16      2433
17      2033
18       925
19      2658
20       187
21      1234
22      1353
23      6287
24      2077
25      9034
26      3477
27      9604
28      8328
29      2833
       ...  
770     3261
771     9897
772     6224
773    11820
774    10844
775     5295
776     7030
777     2038
778    17451
779    23814
780     7978
781     5754
782     2391
783     1757
784     2178
785     3657
786       77
787     6128
788     2669
789     2838
790     3940
791     6018
792     4749
793     3253
794     7513
795     2601
796     5426
797     7380
798    13596
799     5225
Name: max_stroke_length, Length: 800, dtype: int64


#### Stroke Similarity

In [14]:
import svg_render_helpers as rdh

These features are tried but reduces model performances:
- draw_duration

In [42]:
# save raw data with added features
raw_data.to_csv(scribble_file)

#### Filter out needed features

In [45]:
# change age from strings to integers
raw_data['age'] = raw_data['age'].str.replace('age','')
pd.to_numeric(raw_data.age)

# transfer categorical columns into integers
raw_data.category = pd.Categorical(raw_data.category)
raw_data['cat_code'] = raw_data.category.cat.codes

feature_cols = ['cat_code', 'age', 'num_strokes', 'num_black_pixels', 'prop_black', 'max_stroke_length']
select_cols = feature_cols[:]
select_cols.append('scribble')
data = raw_data.loc[:, select_cols]

print data.shape

(800, 7)


### Set a classification model

In [48]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier  
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# split data into train and test sets
train, test = train_test_split(data, test_size=0.4)

# get features
train_x = train.loc[:, feature_cols]
test_x = test.loc[:, feature_cols]

# get scribble values
train_y = train.scribble
test_y = test.scribble

print train_x.shape, train_y.shape
print test_x.shape, test_y.shape

# train
logreg = LogisticRegression()
logreg.fit(train_x, train_y)

gbm = GradientBoostingClassifier(min_samples_split=50, min_samples_leaf=10)
gbm.fit(train_x, train_y)

rf = RandomForestClassifier(max_depth=2, random_state=0)
rf.fit(train_x,train_y)

# prediction on the test set
test_pred_log = logreg.predict(test_x)
test_pred_gbm = gbm.predict(test_x)
test_pred_rf = rf.predict(test_x)


(480, 6) (480,)
(320, 6) (320,)


### Evaluate the model

In [49]:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

# evaluate the model
# f1 = f1_score(test_y, test_pred, average='micro')
# print 'f1 score: ', f1

final_test_pred = np.zeros(len(test_y))
                           
for i, v in enumerate(test_pred_log):
    if v==1 or test_pred_gbm[i]==1:
        final_test_pred[i] = 1
                           
target_names = ['0', '1']
print 'logistic regression'
print(classification_report(test_y, test_pred_log, target_names=target_names))

print 'gradient boosting classifer'
print(classification_report(test_y, test_pred_gbm, target_names=target_names))

print 'random forest'
print(classification_report(test_y, test_pred_rf, target_names=target_names))
                           
print 'combination'
print(classification_report(test_y, final_test_pred, target_names=target_names))


logistic regression
             precision    recall  f1-score   support

          0       0.94      0.97      0.96       287
          1       0.65      0.45      0.54        33

avg / total       0.91      0.92      0.91       320

gradient boosting classifer
             precision    recall  f1-score   support

          0       0.96      0.98      0.97       287
          1       0.74      0.61      0.67        33

avg / total       0.93      0.94      0.93       320

random forest
             precision    recall  f1-score   support

          0       0.94      0.98      0.96       287
          1       0.74      0.42      0.54        33

avg / total       0.92      0.93      0.92       320

combination
             precision    recall  f1-score   support

          0       0.96      0.96      0.96       287
          1       0.64      0.64      0.64        33

avg / total       0.93      0.93      0.93       320

